In [2]:
!pip install torch transformers bitsandbytes accelerate peft vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.3/228.3 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import pandas as pd
from openai import OpenAI
import json
import time
import warnings
warnings.filterwarnings('ignore')

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from huggingface_hub import HfApi

# Your Hugging Face token
hf_token = ""

# Step 1: Load the Base Model and Tokenizer
base_model_name = "microsoft/Phi-3-mini-4k-instruct"  # The base model
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load the base model in 8-bit precision
model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype=torch.float16, device_map="auto")

# Step 2: Load the LoRA Adapter
adapter_model_name = "Sabyasachi/phi-3-mini-4k-finetuned-terraform"  # LoRA adapter on Hugging Face
adapter_config = PeftConfig.from_pretrained(adapter_model_name)
model = PeftModel.from_pretrained(model, adapter_model_name)

# Step 3: Merge LoRA Adapter with Base Model
# This applies the LoRA weights to the base model
model = model.merge_and_unload()

# Step 4: Save the Entire Model (Base + LoRA) to Hugging Face Hub
save_directory = "./phi-3-mini-4k-finetuned-terraform-full"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Push the full model to Hugging Face Hub
api = HfApi()
api.create_repo(repo_id="Sabyasachi/phi-3-mini-4k-finetuned-terraform-full", exist_ok=True)
api.upload_folder(
    folder_path=save_directory,
    repo_id="Sabyasachi/phi-3-mini-4k-finetuned-terraform-full",  # Change to your Hugging Face repo
    repo_type="model",
    token=hf_token
)

print("Full model (base + LoRA) uploaded to Hugging Face Hub!")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Full model (base + LoRA) uploaded to Hugging Face Hub!


In [8]:
os.environ['VLLM_ALLOW_LONG_MAX_MODEL_LEN'] = '1'
model_id = "Sabyasachi/phi-3-mini-4k-finetuned-terraform-full"
#llm = LLM(model=model_id, max_model_len=16240)
llm = LLM(model=model_id, max_model_len=None)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

WARNING 09-30 01:22:39 config.py:1778] User-specified max_model_len (16240) is greater than the derived max_model_len (max_position_embeddings=4096 or model_max_length=None in model's config.json). This may lead to incorrect model outputs or CUDA errors. Make sure the value is correct and within the model context size.
INFO 09-30 01:22:39 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='Sabyasachi/phi-3-mini-4k-finetuned-full', speculative_config=None, tokenizer='Sabyasachi/phi-3-mini-4k-finetuned-full', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=16240, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-30 01:26:38 model_runner.py:1025] Loading model weights took 7.1183 GB
INFO 09-30 01:26:42 gpu_executor.py:122] # GPU blocks: 1992, # CPU blocks: 682


OutOfMemoryError: CUDA out of memory. Tried to allocate 374.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 6.88 MiB is free. Process 43717 has 22.15 GiB memory in use. Of the allocated memory 21.91 GiB is allocated by PyTorch, and 16.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def chatGPT4oresponse(ques):

    for _ in range(MAX_RETRIES):
        try:
            response = client.chat.completions.create(
                model='gpt-4o',
                messages=[
                    {"role": "user", "content": "{}".format(ques)},
                ],
                temperature=0,
            )
            print(response.choices[0].message.content)
            return(response.choices[0].message.content)
        except Exception as e:
            if _ == MAX_RETRIES - 1:  # Last iteration
                error = "An error occurred: {}\n".format(e)
                return error
            time.sleep(SLEEP_DURATION)  # Sleep before retrying

def base_gpt4_model(ques):
    for _ in range(MAX_RETRIES):
        try:
            response = client.chat.completions.create(
                model='gpt-4-turbo',
                messages=[
                    {"role": "user", "content": "{}".format(ques)},
                ],
                temperature=0,
            )
            return(response.choices[0].message.content)
        except Exception as e:
            if _ == MAX_RETRIES - 1:  # Last iteration
                return "An error occurred: {}".format(e)
            time.sleep(SLEEP_DURATION)  # Sleep before retrying

def generate_terraform_code(user_query,cloud_provider):
    instruction = "Generate Terraform code based on the provided requirements."
    prompt="""You are a Terraform consultant with expertise in AWS, Azure, and GCP. Your task is to develop a Terraform configuration based on the provided user query for the specified cloud provider.

    - *User Query:* {}
    - *Cloud Provider:* {}

    This Terraform configuration must include modules for essential components and design the modules to be reusable and easy to maintain.
    Adhere to the industry best practices for security and efficiency throughout the configuration. The output Terraform config should be in a structured format that is easy to understand and implement.
    It should meet the Terraform generated code quality considerations(Correctness,Idempotency,Security,Performance and Efficiency,
    Scalability,Documentation,Compliance and Modularity and Reusability)"""
    updated_prompt=prompt.format(user_query,cloud_provider)
    chat = [
        #{"role": "system", "content": instruction},
        {"role": "user", "content": updated_prompt}
    ]

    formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    return formatted_prompt


In [ ]:
df=pd.read_excel('Research_Analysis_Base_data_filtered_fine_tune_inference.xlsx')
del df['Unnamed: 0']
print(len(df))

In [ ]:
df.head()

In [ ]:
prompt_lst=[]
for index, row in df.iterrows():
    user_query=row['user_query']
    cloud_provider=row['Cloud_provider']
    #print("user query is {}".format(user_query))
    try:
        updated_prompt=generate_terraform_code(user_query,cloud_provider)
        prompt_lst.append(updated_prompt)
    except Exception as error:
        print(error)

In [ ]:
prompt_lst[0]

In [ ]:
sampling_params = SamplingParams(temperature=0.7, top_p=0.95, max_tokens=2500, stop=['[/SYS]', "<s>", "</s>","<|end|>"])
outputs= llm.generate(prompt_lst, sampling_params)
terraform_code = [output.outputs[0].text.strip() for output in outputs]

In [ ]:
df['microsoft:Phi-3-mini-4k-instruct_fine-tuned_code_generation']=terraform_code

In [ ]:
print(df.iloc[4]['microsoft:Phi-3-mini-4k-instruct_fine-tuned_code_generation'])

In [ ]:
df.to_excel("Research_Analysis_Base_data_filtered_new_microsoft_Phi-3-mini-4k-instruct_fine_tuned_code_generation2.xlsx")